In [1]:
# This Code is currently Broken and needs to be updated to reflect new path structure
# - Kelson Shilling-Scrivo 2019



In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv




In [3]:
# Change to your filepath to data and uncomment if NFIRS_2009_2016_geocoded_with_tract.csv not in the main path 
os.chdir('../Data/Raw')

In [4]:
os.getcwd()

'/Users/kelson/Documents/DataKind/Red_Cross_Fire/Data/Raw'

In [5]:
# load the data 

# all code is located in the Google Drive under 03_geocodes 
#https://drive.google.com/open?id=1n939Tckv0SFBSiABwIH64M6FSDJHt0eF

# NFIR = automatically geocoded data
# NFIR_2 = handcoded data from Jake of some of the missing rows from NFIR
# NFIR_orig = the original NFIRS dataset 


NFIR = pd.read_csv('NFIRS_2009_2016_geocoded_with_tract.csv',
                   encoding='latin_1',
                   index_col=0,
                   low_memory=False)

NFIR_2 = pd.read_csv('NFIRS_2009_2016_FIXED_unique_geocodes_census_tracts.csv',
                   encoding='latin_1',
                   index_col=0,
                   low_memory=False)

NFIR_orig = pd.read_csv('NFIRS_2009_2016.csv',
                   encoding='latin_1',
                   index_col=0,
                   low_memory=False)



/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
row_count = sum(1 for row in csv.reader( open('NFIRS_2009_2016_geocoded_with_tract.csv' ) ))
row_count2 = sum(1 for row in csv.reader( open('NFIRS_2009_2016_FIXED_unique_geocodes_census_tracts.csv' ) ))

assert(row_count  - NFIR.shape[0] )
assert(row_count2 - NFIR_2.shape[0] )

print(row_count, row_count2)

1959106 71990


In [61]:
##Cleaning 

NFIR.dropna(inplace = True)

# Duplicate checking
dup_flag = set(NFIR.index.values.tolist() ) & set( NFIR_2.index.values.tolist() )

assert( dup_flag == set() )

print(dup_flag)




set()


In [35]:
NFIR_2.loc[:,'X.summary.'].value_counts()

No Match                  35564
Match-Mailable            20644
Match-Inactive            12552
No Match - PO Box Only     2259
Match-Vacant                970
Name: X.summary., dtype: int64

In [38]:
# munging 

#  recreate the original address catagory to be compatible with the original NFIR file 
NFIR_2['ORIGINAL_ADDRESS'] = NFIR_2['Address'] +', ' + NFIR_2['City'] +', ' + NFIR_2['STATE'] + ', ' + NFIR_2['ZIP']

# set ID as index variable 
NFIR_2 = NFIR_2.set_index('ID')


# Appending All matching NFIR_2 Data to NFIR to create the full geocoded NFIRS dataset
NFIR_all = NFIR.append(NFIR_2.loc[:,[ 'ORIGINAL_ADDRESS',
                                     'X','Y','STATE',
                                     'state_fips','COUNTYFP',
                                     'TRACTCE','GEOID']], sort = True ) 

KeyError: "None of [Index(['X.summary.'], dtype='object')] are in the [columns]"

In [56]:
Index =  NFIR_2.loc[:,'X.summary.'].str.contains('Match-')

In [59]:
# we only care about adding matched addresses 
NFIR.index

Index(['rcp2_id1869', 'rcp2_id9391', 'rcp2_id9392', 'rcp2_id9393',
       'rcp2_id9394', 'rcp2_id9395', 'rcp2_id9396', 'rcp2_id9397',
       'rcp2_id1861', 'rcp2_id9388',
       ...
       'rcp2_id669947', 'rcp2_id664175', 'rcp2_id664182', 'rcp2_id669962',
       'rcp2_id669961', 'rcp2_id665719', 'rcp2_id669989', 'rcp2_id669983',
       'rcp2_id664138', 'rcp2_id669990'],
      dtype='object', name='ID', length=1566922)

In [21]:
NFIR_all.tail()

,COUNTYFP,FUNCSTAT,GEOID,MATCH,MATCHED_ADDRESS,MATCH_TYPE,MTFCC,NAME,NAMELSAD,ORIGINAL_ADDRESS,STATE,TIGER_LINE_ID,TIGER_LINE_SIDE,TRACTCE,X,Y,state_fips
ID,,,,,,,,,,,,,,,,,
rcp2_id1753662,11.0,NaN,4.201101e+10,NaN,NaN,NaN,NaN,NaN,NaN,"ZIEGLERS RD, BERNTOWNSHIP OF, PA, 19605",PA,NaN,NaN,12700.0,-75.93677,40.39070,42.0
rcp2_id1332366,215.0,NaN,2.921548e+10,NaN,NaN,NaN,NaN,NaN,NaN,"ZIMMERMAN, CABOOL, MO, 65689",MO,NaN,NaN,480400.0,-92.09128,37.15987,29.0
rcp2_id1332364,215.0,NaN,2.921548e+10,NaN,NaN,NaN,NaN,NaN,NaN,"ZIMMERMANN, CABOOL, MO, 65689",MO,NaN,NaN,480400.0,-92.09128,37.15987,29.0
rcp2_id1532504,275.0,NaN,2.275000e+09,NaN,NaN,NaN,NaN,NaN,NaN,"ZIMOVIA, WRANGELL, AK, 99929",AK,NaN,NaN,300.0,-132.38707,56.47183,2.0
rcp2_id1924543,167.0,NaN,3.916702e+10,NaN,NaN,NaN,NaN,NaN,NaN,"ZION RIDGE RD, NEWPORT, OH, 45768",OH,NaN,NaN,21200.0,-81.26043,39.39918,39.0


In [10]:
NFIR_orig['ID'] = 'rcp2_id' + NFIR_orig.index.astype('str')

In [11]:
# adding ID to Original data set 

NFIR_orig = NFIR_orig.set_index('ID')


# merging Geocoded dataset with original Dataset
NFIR_merged_final = NFIR_orig.merge(NFIR_all,how='left',left_index = True, right_index = True)

In [75]:
# row duplication check 
NFIR_merged_final.head()



,state,fdid,inc_no,city,street,zip5,lat,lon,fips,inc_date,...,NAME,NAMELSAD,ORIGINAL_ADDRESS,STATE,TIGER_LINE_ID,TIGER_LINE_SIDE,TRACTCE,X,Y,state_fips
ID,,,,,,,,,,,,,,,,,,,,,
rcp2_id1,AK,11100,211,JUNEAU,1700 ANGUS WAY,99801,NaN,NaN,NaN,1/3/2009 0:00:00,...,5.0,Census Tract 5,"1700 ANGUS WAY, JUNEAU, AK, 99801",AK,190964136.0,R,500.0,-134.42440,58.306507,2.0
rcp2_id2,AK,11100,470,JUNEAU,3221 PIONEER AVE,99801,NaN,NaN,NaN,1/6/2009 0:00:00,...,6.0,Census Tract 6,"3221 PIONEER AVE, JUNEAU, AK, 99801",AK,190966361.0,R,600.0,-134.43314,58.292686,2.0
rcp2_id3,AK,11100,556,JUNEAU,10231 HERON WAY,99801,NaN,NaN,NaN,1/6/2009 0:00:00,...,1.0,Census Tract 1,"10231 HERON WAY, JUNEAU, AK, 99801",AK,190962538.0,R,100.0,-134.60330,58.402610,2.0
rcp2_id4,AK,11100,1315,JUNEAU,6590 GLACIER HWY,99801,NaN,NaN,NaN,1/13/2009 0:00:00,...,4.0,Census Tract 4,"6590 GLACIER HWY, JUNEAU, AK, 99801",AK,190963741.0,R,400.0,-134.51932,58.358955,2.0
rcp2_id5,AK,11100,2151,JUNEAU,6590 GLACIER HWY,99801,NaN,NaN,NaN,1/21/2009 0:00:00,...,4.0,Census Tract 4,"6590 GLACIER HWY, JUNEAU, AK, 99801",AK,190963741.0,R,400.0,-134.51932,58.358955,2.0


In [13]:
NFIR_merged_final.to_csv('..//Transformed//NFIRS_2009_2016_Combined_Census_Tract.csv')


,ORIGINAL_ADDRESS,MATCH,MATCH_TYPE,MATCHED_ADDRESS,X,Y,TIGER_LINE_ID,TIGER_LINE_SIDE,STATE,state_fips,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT
ID,,,,,,,,,,,,,,,,,
rcp2_id1826093,"6843 Obrien ST, Anchorage, AK, 99507",Match,Exact,"6843 OBRIEN ST, ANCHORAGE, AK,99507",-149.83640,61.158850,625154482.0,R,AK,2.0,20.0,2601.0,2.020003e+09,26.01,Census Tract 26.01,G5020,S
rcp2_id1521980,"2588 SILVER ST, NORTH POLE, AK, 99705",Match,Exact,"2588 SILVER ST, NORTH POLE, AK,99705",-147.24670,64.749620,190925385.0,R,AK,2.0,90.0,1500.0,2.090002e+09,15.00,Census Tract 15,G5020,S
rcp2_id972993,"2168 YELLOWSNOW RD, Fairbanks, AK,99709",Match,Exact,"2168 YELLOWSNOW RD, FAIRBANKS, AK,99709",-147.86130,64.923040,190910379.0,R,AK,2.0,90.0,1900.0,2.090002e+09,19.00,Census Tract 19,G5020,S
rcp2_id1607348,"709 Hemlock ST, Kodiak, AK, 99615",Match,Exact,"709 HEMLOCK ST, KODIAK, AK,99615",-152.40090,57.795010,193545384.0,L,AK,2.0,150.0,300.0,2.150000e+09,3.00,Census Tract 3,G5020,S
rcp2_id1607347,"2907 Turnagain ST, Anchorage, AK, 99517",Match,Exact,"2907 TURNAGAIN ST, ANCHORAGE, AK,99517",-149.93530,61.194230,190794794.0,L,AK,2.0,20.0,2202.0,2.020002e+09,22.02,Census Tract 22.02,G5020,S
rcp2_id98,"1200 Columbine CT, Anchorage, AK, 99508",Match,Exact,"1200 COLUMBINE CT, ANCHORAGE, AK, 99508",-149.81776,61.210674,605168076.0,R,AK,2.0,20.0,901.0,2.020001e+09,9.01,Census Tract 9.01,G5020,S
rcp2_id99,"724 N Flower ST, Anchorage, AK, 99508",Match,Exact,"724 N FLOWER ST, ANCHORAGE, AK, 99508",-149.80653,61.229890,190801497.0,L,AK,2.0,20.0,600.0,2.020001e+09,6.00,Census Tract 6,G5020,S
rcp2_id92,"95 ANDERSON ST, ALAKANUK, AK, 99554",Match,Exact,"95 Anderson St, ALAKANUK, AK, 99554",-164.63490,62.684950,191268875.0,L,AK,2.0,158.0,100.0,2.158000e+09,1.00,Census Tract 1,G5020,S
rcp2_id234309,"2221 Muldoon RD, Anchorage, AK, 99504",Match,Exact,"2221 MULDOON RD, ANCHORAGE, AK, 99504",-149.73346,61.197445,190803386.0,L,AK,2.0,20.0,1731.0,2.020002e+09,17.31,Census Tract 17.31,G5020,S
